In [1]:
import sys, os, re, csv, codecs
import numpy as np
import pandas as pd

import pickle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPooling1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import Callback
from keras.layers import AveragePooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import concatenate
from keras.layers import SpatialDropout1D
from keras.layers import GRU

Using TensorFlow backend.


In [2]:
repl = {
    "&lt;3": " good ", ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

In [3]:
def save_variable_pickle(var, name, model = False):
    filename = name + '.pickle'
    if model:
        joblib.dump(var, filename = filename)
    else:  
        with open(filename, 'wb') as f:
            pickle.dump(var, f, pickle.HIGHEST_PROTOCOL)

def load_variable_pickle(name, model = False):
    filename = name
    if model:
        return joblib.load(filename = filename)
    else:
        with open(filename, 'rb') as f:
            return pickle.load(f)
        

In [4]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [5]:
TRAIN_DATA_FILE = '/home/yichen/Desktop/toxic/input/train.csv'
TEST_DATA_FILE = '/home/yichen/Desktop/toxic/input/test.csv'
glove = '/home/yichen/Desktop/DL_pretrain/glove.840B.300d.txt'

In [6]:
embed_size = 300 # how big is each word vector
max_features = 30000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100

In [7]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

#x_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [8]:
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data

trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = trate
test["comment_text"] = tete
del trate, tete
train.drop(["new_comment_text"], axis=1, inplace=True)
test.drop(["new_comment_text"], axis=1, inplace=True)

In [9]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train) + list(list_sentences_test))

list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

x_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
x_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [10]:
x_train.shape, x_test.shape

((159571, 100), (153164, 100))

In [11]:
glove_preprocess = load_variable_pickle('glove_preprocessing.pickle')

In [12]:
all_embs = np.stack(glove_preprocess.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(-0.0058391555, 0.4878261)

In [13]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = glove_preprocess.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [14]:
x_train.shape, y_train.shape

((159571, 100), (159571, 6))

In [15]:
x_train_features, x_test_features, y_train_features, y_test_features = train_test_split(x_train, y_train, test_size = 0.5, shuffle = False)

In [16]:
kf = KFold(n_splits=5, random_state=1023)
county = 0
for train_index, test_index in kf.split(x_train_features):
    if county == 0:
        train_index0 = train_index
        test_index0 = test_index
    elif county == 1:
        train_index1 = train_index
        test_index1 = test_index
    elif county == 2:
        train_index2 = train_index
        test_index2 = test_index
    elif county == 3:   
        train_index3 = train_index
        test_index3 = test_index
    elif county == 4:
        train_index4 = train_index
        test_index4 = test_index
    county = county + 1
#x_train_features, x_test_features, y_train_features, y_test_features
x_train0, y_train0, x_test0 = x_train_features[train_index0], y_train_features[train_index0], x_test_features[test_index0]
x_train1, y_train1, x_test1 = x_train_features[train_index1], y_train_features[train_index1], x_test_features[test_index1]
x_train2, y_train2, x_test2 = x_train_features[train_index2], y_train_features[train_index2], x_test_features[test_index2]
x_train3, y_train3, x_test3 = x_train_features[train_index3], y_train_features[train_index3], x_test_features[test_index3]
x_train4, y_train4, x_test4 = x_train_features[train_index4], y_train_features[train_index4], x_test_features[test_index4]


In [21]:
def GRU_858model():    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [22]:
gru_model = GRU_858model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train0, y_train0, train_size=0.95,)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [23]:
gru_cv0 = gru_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
y_pred0 = gru_model.predict(x_test0, batch_size=1024)

Train on 60636 samples, validate on 3192 samples
Epoch 1/2
 3072/60636 [>.............................] - ETA: 4:49 - loss: 0.1538 - acc: 0.9558

KeyboardInterrupt: 

In [19]:
gru_model = GRU_858model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train1, y_train1, train_size=0.95,)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [20]:
gru_cv1 = gru_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
y_pred1 = gru_model.predict(x_test1, batch_size=1024)

Train on 121274 samples, validate on 6383 samples
Epoch 1/2
121274/121274 [==============================] - 430s 4ms/step - loss: 0.0511 - acc: 0.9814 - val_loss: 0.0433 - val_acc: 0.9836

 ROC-AUC - epoch: 1 - score: 0.985241 

Epoch 2/2
121274/121274 [==============================] - 558s 5ms/step - loss: 0.0384 - acc: 0.9852 - val_loss: 0.0449 - val_acc: 0.9829

 ROC-AUC - epoch: 2 - score: 0.985964 



In [21]:
gru_model = GRU_858model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train2, y_train2, train_size=0.95,)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [22]:
gru_cv2 = gru_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
y_pred2 = gru_model.predict(x_test2, batch_size=1024)

Train on 121274 samples, validate on 6383 samples
Epoch 1/2
121274/121274 [==============================] - 578s 5ms/step - loss: 0.0509 - acc: 0.9815 - val_loss: 0.0473 - val_acc: 0.9809

 ROC-AUC - epoch: 1 - score: 0.987699 

Epoch 2/2
121274/121274 [==============================] - 424s 4ms/step - loss: 0.0384 - acc: 0.9852 - val_loss: 0.0453 - val_acc: 0.9819

 ROC-AUC - epoch: 2 - score: 0.987594 



In [23]:
gru_model = GRU_858model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train3, y_train3, train_size=0.95,)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [24]:
gru_cv3 = gru_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
y_pred3 = gru_model.predict(x_test3, batch_size=1024)

Train on 121274 samples, validate on 6383 samples
Epoch 1/2
121274/121274 [==============================] - 428s 4ms/step - loss: 0.0502 - acc: 0.9818 - val_loss: 0.0440 - val_acc: 0.9830

 ROC-AUC - epoch: 1 - score: 0.985589 

Epoch 2/2
121274/121274 [==============================] - 427s 4ms/step - loss: 0.0384 - acc: 0.9849 - val_loss: 0.0440 - val_acc: 0.9829

 ROC-AUC - epoch: 2 - score: 0.986629 



In [25]:
gru_model = GRU_858model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train4, y_train4, train_size=0.95,)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [26]:
gru_cv4 = gru_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
y_pred4 = gru_model.predict(x_test4, batch_size=1024)

Train on 121274 samples, validate on 6383 samples
Epoch 1/2
121274/121274 [==============================] - 419s 3ms/step - loss: 0.0508 - acc: 0.9814 - val_loss: 0.0429 - val_acc: 0.9835

 ROC-AUC - epoch: 1 - score: 0.983100 

Epoch 2/2
121274/121274 [==============================] - 411s 3ms/step - loss: 0.0383 - acc: 0.9850 - val_loss: 0.0422 - val_acc: 0.9833

 ROC-AUC - epoch: 2 - score: 0.982665 



In [44]:
gru_model = GRU_858model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95,)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [45]:
gru_metab = gru_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
test_meta_gru = gru_model.predict(x_test, batch_size=1024)

Train on 151592 samples, validate on 7979 samples
Epoch 1/2
151592/151592 [==============================] - 567s 4ms/step - loss: 0.0498 - acc: 0.9817 - val_loss: 0.0402 - val_acc: 0.9842

 ROC-AUC - epoch: 1 - score: 0.989278 

Epoch 2/2
151592/151592 [==============================] - 552s 4ms/step - loss: 0.0383 - acc: 0.9851 - val_loss: 0.0403 - val_acc: 0.9840

 ROC-AUC - epoch: 2 - score: 0.989321 



In [27]:
def lstm_70model():    
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
    x = Dropout(0.1)(x)
    x = Bidirectional(LSTM(200, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam',  metrics = ['accuracy'])
    return model

In [28]:
lstm_model = lstm_70model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train0, y_train0, train_size=0.95, random_state=1023)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [29]:
lstm_cv0 = lstm_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
lstm_y_pred0 = lstm_model.predict(x_test0, batch_size=1024)

Train on 121273 samples, validate on 6383 samples
Epoch 1/2
121273/121273 [==============================] - 605s 5ms/step - loss: 0.0520 - acc: 0.9812 - val_loss: 0.0426 - val_acc: 0.9842

 ROC-AUC - epoch: 1 - score: 0.984570 

Epoch 2/2
121273/121273 [==============================] - 642s 5ms/step - loss: 0.0392 - acc: 0.9847 - val_loss: 0.0437 - val_acc: 0.9840

 ROC-AUC - epoch: 2 - score: 0.984872 



In [30]:
lstm_model = lstm_70model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train1, y_train1, train_size=0.95, random_state=1023)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [31]:
lstm_cv1 = lstm_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
lstm_y_pred1 = lstm_model.predict(x_test1, batch_size=1024)

Train on 121274 samples, validate on 6383 samples
Epoch 1/2
121274/121274 [==============================] - 708s 6ms/step - loss: 0.0514 - acc: 0.9813 - val_loss: 0.0456 - val_acc: 0.9820

 ROC-AUC - epoch: 1 - score: 0.988200 

Epoch 2/2
121274/121274 [==============================] - 620s 5ms/step - loss: 0.0394 - acc: 0.9847 - val_loss: 0.0437 - val_acc: 0.9833

 ROC-AUC - epoch: 2 - score: 0.986701 



In [32]:
lstm_model = lstm_70model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train2, y_train2, train_size=0.95, random_state=1023)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [33]:
lstm_cv2 = lstm_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
lstm_y_pred2 = lstm_model.predict(x_test2, batch_size=1024)

Train on 121274 samples, validate on 6383 samples
Epoch 1/2
121274/121274 [==============================] - 696s 6ms/step - loss: 0.0517 - acc: 0.9813 - val_loss: 0.0449 - val_acc: 0.9820

 ROC-AUC - epoch: 1 - score: 0.985824 

Epoch 2/2
121274/121274 [==============================] - 633s 5ms/step - loss: 0.0396 - acc: 0.9845 - val_loss: 0.0451 - val_acc: 0.9825

 ROC-AUC - epoch: 2 - score: 0.986196 



In [34]:
lstm_model = lstm_70model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train3, y_train3, train_size=0.95, random_state=1023)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [35]:
lstm_cv3 = lstm_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
lstm_y_pred3 = lstm_model.predict(x_test3, batch_size=1024)

Train on 121274 samples, validate on 6383 samples
Epoch 1/2
121274/121274 [==============================] - 734s 6ms/step - loss: 0.0515 - acc: 0.9813 - val_loss: 0.0409 - val_acc: 0.9840

 ROC-AUC - epoch: 1 - score: 0.986770 

Epoch 2/2
121274/121274 [==============================] - 870s 7ms/step - loss: 0.0393 - acc: 0.9845 - val_loss: 0.0411 - val_acc: 0.9841

 ROC-AUC - epoch: 2 - score: 0.989230 



In [36]:
lstm_model = lstm_70model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train4, y_train4, train_size=0.95, random_state=1023)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [37]:
lstm_cv4 = lstm_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
lstm_y_pred4 = lstm_model.predict(x_test4, batch_size=1024)

Train on 121274 samples, validate on 6383 samples
Epoch 1/2
121274/121274 [==============================] - 638s 5ms/step - loss: 0.0513 - acc: 0.9814 - val_loss: 0.0415 - val_acc: 0.9843

 ROC-AUC - epoch: 1 - score: 0.985604 

Epoch 2/2
121274/121274 [==============================] - 634s 5ms/step - loss: 0.0391 - acc: 0.9846 - val_loss: 0.0410 - val_acc: 0.9848

 ROC-AUC - epoch: 2 - score: 0.985085 



In [38]:
lstm_y_pred = lstm_y_pred0
lstm_y_pred = np.concatenate((lstm_y_pred, lstm_y_pred1), axis = 0)
lstm_y_pred = np.concatenate((lstm_y_pred, lstm_y_pred2), axis = 0)
lstm_y_pred = np.concatenate((lstm_y_pred, lstm_y_pred3), axis = 0)
lstm_y_pred = np.concatenate((lstm_y_pred, lstm_y_pred4), axis = 0)
gru_y_pred = y_pred0
gru_y_pred = np.concatenate((gru_y_pred, y_pred1), axis = 0)
gru_y_pred = np.concatenate((gru_y_pred, y_pred2), axis = 0)
gru_y_pred = np.concatenate((gru_y_pred, y_pred3), axis = 0)
gru_y_pred = np.concatenate((gru_y_pred, y_pred4), axis = 0)
gru_y_pred.shape, lstm_y_pred.shape

((159571, 6), (159571, 6))

In [41]:
lstm_model = lstm_70model()
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95,)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [43]:
lstm_metab = lstm_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
test_meta_lstm = lstm_model.predict(x_test, batch_size=1024)

Train on 151592 samples, validate on 7979 samples
Epoch 1/2
151592/151592 [==============================] - 881s 6ms/step - loss: 0.0505 - acc: 0.9815 - val_loss: 0.0414 - val_acc: 0.9845

 ROC-AUC - epoch: 1 - score: 0.989182 

Epoch 2/2
151592/151592 [==============================] - 799s 5ms/step - loss: 0.0393 - acc: 0.9846 - val_loss: 0.0397 - val_acc: 0.9850

 ROC-AUC - epoch: 2 - score: 0.989577 



In [18]:
from keras.layers import Conv1D, Conv2D, Flatten, MaxPooling1D

In [47]:
save_variable_pickle(test_meta_gru, 'test_meta_gru')
save_variable_pickle(test_meta_lstm, 'test_meta_lstm')
save_variable_pickle(partB_meta_gru, 'partB_meta_gru')
save_variable_pickle(partB_meta_lstm, 'partB_meta_lstm')
save_variable_pickle(gru_y_pred, 'gru_y_pred')
save_variable_pickle(lstm_y_pred, 'lstm_y_pred')

In [51]:
from keras.layers import Conv1D, Conv2D, Flatten, MaxPooling1D

In [52]:
embedding_layer = Embedding(len(word_index) + 1, max_features, input_length=embed_size,  weights=[embedding_matrix], trainable=False)

In [53]:
def cnn_model():    
    sequence_input = Input(shape=(maxlen,))
    embedded_sequence = embedding_layer(sequence_input)
    # = Dropout(0.1)(embedded_sequence)
    x = Conv1D(100,3, activation = 'relu')(embedded_sequence)
    x = Dropout(0.1)(x)
    x = MaxPooling1D(3)(x)
    # = Conv1D(128, 5, activation = 'relu')(x)
    # = Dropout(0.1)(x)
    # = MaxPooling1D(5)(x)
    #x = Conv1D(128, 5, activation = 'relu')(x)
    #x = MaxPooling1D(35)(x)
    x = Flatten()(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs = sequence_input, outputs = x)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam',  metrics = ['accuracy'])
    return model

In [55]:
cnn_model = cnn_model()
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 300, 30000)        1184364000
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 298, 100)          9000100   
_________________________________________________________________
dropout_11 (Dropout)         (None, 298, 100)          0         
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 99, 100)           0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 9900)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                633664    
__________

In [56]:
batch_size = 32
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train0, y_train0, train_size=0.95, random_state=1023)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

/home/yichen/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [57]:
cnn_cv0 = cnn_model.fit(X_tra, y_tra, batch_size = 32, epochs = 2, validation_data = (X_val, y_val), callbacks=[RocAuc], verbose=1)
#cnn_y_pred0 = cnn_model.predict(x_test0, batch_size=1024)

Train on 60636 samples, validate on 3192 samples
Epoch 1/2


FailedPreconditionError: Attempting to use uninitialized value embedding_3/embeddings
	 [[Node: embedding_3/embeddings/read = Identity[T=DT_FLOAT, _class=["loc:@embedding_3/embeddings"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_3/embeddings)]]
	 [[Node: metrics_6/acc/Mean_1/_593 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_724_metrics_6/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'embedding_3/embeddings/read', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yichen/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/y